## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

random.seed(1234)
train_set, val_set = train_test_split(nltk_data,test_size=0.05)

# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

# number of tags
T = set([pair[1] for pair in train_tagged_words])
print (T)

# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')

# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

{'.', 'NUM', 'NOUN', 'ADJ', 'VERB', 'X', 'ADP', 'PRT', 'CONJ', 'ADV', 'DET', 'PRON'}


,.,NUM,NOUN,ADJ,VERB,X,ADP,PRT,CONJ,ADV,DET,PRON
.,0.093756,0.080220,0.220809,0.045208,0.088251,0.026890,0.092944,0.002346,0.058202,0.052247,0.173705,0.065331
NUM,0.116929,0.182684,0.357334,0.034514,0.018744,0.206486,0.035406,0.026778,0.013389,0.002975,0.003273,0.001488
NOUN,0.238952,0.009618,0.263725,0.012277,0.147473,0.028963,0.177639,0.043681,0.042588,0.017378,0.013152,0.004554
ADJ,0.065367,0.020970,0.697248,0.068152,0.012123,0.021298,0.076343,0.010976,0.016874,0.004915,0.005079,0.000655
VERB,0.034202,0.022956,0.109508,0.065147,0.169769,0.218163,0.090817,0.031798,0.005429,0.082131,0.134559,0.035520
X,0.160326,0.002558,0.063459,0.017104,0.204124,0.075448,0.144182,0.185902,0.009431,0.025895,0.054987,0.056586
ADP,0.039157,0.063205,0.320600,0.107470,0.008300,0.034050,0.017131,0.001383,0.000426,0.014152,0.324963,0.069164
PRT,0.043109,0.056826,0.247877,0.083932,0.399739,0.013717,0.021228,0.001633,0.002286,0.010124,0.100914,0.018615
CONJ,0.034998,0.042464,0.350910,0.118059,0.155390,0.007933,0.049930,0.005133,0.000467,0.055996,0.120859,0.057863
ADV,0.136603,0.030612,0.031929,0.127716,0.342989,0.023041,0.118828,0.014812,0.007242,0.081633,0.069783,0.014812


In [5]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Build the vanilla Viterbi based POS tagger

In [6]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [7]:
# list of tagged words
val_run_base = [tup for sent in val_set for tup in sent]

# list of untagged words
val_tagged_words = [tup[0] for sent in val_set for tup in sent]
val_tagged_words

['They',
 'must',
 'figure',
 'that',
 'justice',
 'has',
 '*-2',
 'to',
 'get',
 'done',
 '*-3',
 'by',
 'somebody',
 ',',
 'but',
 'know',
 '0',
 'it',
 'wo',
 "n't",
 'be',
 'done',
 '*-1',
 'by',
 'Congress',
 '.',
 'The',
 'vicar',
 ',',
 'W.D.',
 'Jones',
 ',',
 'refuses',
 '*-1',
 'to',
 'talk',
 'about',
 'it',
 ',',
 '*-1',
 'saying',
 '0',
 'it',
 'would',
 '``',
 'reopen',
 'the',
 'wound',
 '.',
 "''",
 'Other',
 'rumored',
 'takeover',
 'and',
 'restructuring',
 'candidates',
 '0',
 '*T*-2',
 'to',
 'attract',
 'buyers',
 'included',
 'Woolworth',
 ',',
 'which',
 '*T*-1',
 'went',
 'up',
 '1',
 '3\\/4',
 'to',
 '59',
 '1\\/2',
 ';',
 'Avon',
 'Products',
 ',',
 'up',
 '1',
 '3\\/4',
 'to',
 '29',
 '1\\/4',
 ';',
 'Paramount',
 'Communications',
 ',',
 'up',
 '2',
 'to',
 '57',
 '7\\/8',
 ',',
 'and',
 'Ferro',
 ',',
 'up',
 '2',
 '5\\/8',
 'to',
 '28',
 '3\\/4',
 '.',
 'The',
 'campaign',
 'has',
 'blamed',
 'these',
 'reporting',
 'problems',
 'on',
 'computer',
 'errors

### Solve the problem of unknown words

In [8]:
import re
import collections

# Addtional check for the right tagger
def checkNumberTagger(word):
    # check if it is a number
    pattern = "[0-9]+[\:,\,]*[0-9]+"
    if re.match(pattern, word):
        print ("Correcting :: ", word, " to NUM")
        return 'NUM'
    else:
        return 'None'
    
# Instead of usign a random tagger, use biagram to tag based on frequency of the previous tag
def checkBiagramTagger(prev_tag, word, train_bag):
    word_tag_pairs = nltk.bigrams(train_bag)
    tag_seq_noun = list(nltk.FreqDist(a[1] for (a, b) in word_tag_pairs if b[1] == 'NOUN'))
    if prev_tag in tag_seq_noun:
        print ("Correcting :: ", word, " to NOUN when prev tag is ", prev_tag)
        return 'NOUN'
    else:
        return 'None'

# Instead of usign a random tagger, returning the most common tagger for unknown tags
def checkLexicanTagger(word):
    pos_list = nltk.pos_tag([pair[0] for pair in train_tagged_words],'universal')
    pos_counts = collections.Counter((subl[1] for subl in pos_list))
    most_common_tag = pos_counts.most_common(1)[0][0]
    print ("Correcting :: ", word, " with most common tag ", most_common_tag)
    return most_common_tag

# Viterbi Heuristic
def ViterbiNew(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)    
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        # Checking Rule based Tagger in case the transition probabilit is zero. The vanila method was defaulting to first element
        if pmax == 0:
            state_max = checkNumberTagger(word)
            if (state_max == 'None' and len(state) > 0):
                state_max = checkBiagramTagger(state[key-1], word, train_bag)
            if (state_max == 'None'):
                state_max = checkLexicanTagger(word)
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [9]:
# tagging the validation sentences
start = time.time()
tagged_seq = Viterbi(val_tagged_words)
end = time.time()
difference = end-start
print ("Time Taken with Vanila Viterbi on Validation Data ::", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j] 
accuracy_vanila = len(check)/len(tagged_seq)
print ("Accuracy ::", accuracy_vanila)
incorrect_tagged_cases_vanila = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_vanila

Time Taken with Vanila Viterbi on Validation Data :: 323.891578912735
Accuracy :: 0.9080505922505521


[[(',', '.'), (('W.D.', '.'), ('W.D.', 'NOUN'))],
 [(',', '.'), (('refuses', '.'), ('refuses', 'VERB'))],
 [('the', 'DET'), (('wound', 'VERB'), ('wound', 'NOUN'))],
 [('and', 'CONJ'), (('restructuring', 'NOUN'), ('restructuring', 'VERB'))],
 [('included', 'VERB'), (('Woolworth', '.'), ('Woolworth', 'NOUN'))],
 [('Avon', 'NOUN'), (('Products', '.'), ('Products', 'NOUN'))],
 [(';', '.'), (('Paramount', '.'), ('Paramount', 'NOUN'))],
 [('and', 'CONJ'), (('Ferro', '.'), ('Ferro', 'NOUN'))],
 [('computer', 'NOUN'), (('errors', '.'), ('errors', 'NOUN'))],
 [('are', 'VERB'), (('betting', '.'), ('betting', 'VERB'))],
 [('health', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('care', 'NOUN'), (('providers', '.'), ('providers', 'NOUN'))],
 [('and', 'CONJ'), (('that', 'DET'), ('that', 'ADP'))],
 [('of', 'ADP'), (('pregnant', '.'), ('pregnant', 'ADJ'))],
 [('the', 'DET'), (('right', 'NOUN'), ('right', 'ADJ'))],
 [('this', 'DET'), (('minute', 'ADJ'), ('minute', 'NOUN'))],
 [('In', 'ADP'), (('S

In [10]:
# tagging the validation sentences using the modified Viterbi
start = time.time()
tagged_seq = ViterbiNew(val_tagged_words)
end = time.time()
difference = end-start
print ("Time Taken with modified Viterbi on Validation Data ::", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j] 
accuracy_new = len(check)/len(tagged_seq)
print ("Accuracy ::", accuracy_new)
incorrect_tagged_cases_new = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_new

Correcting ::  W.D.  to NOUN when prev tag is  .
Correcting ::  refuses  to NOUN when prev tag is  .
Correcting ::  Woolworth  to NOUN when prev tag is  VERB
Correcting ::  Products  to NOUN when prev tag is  NOUN
Correcting ::  Paramount  to NOUN when prev tag is  .
Correcting ::  Ferro  to NOUN when prev tag is  CONJ
Correcting ::  errors  to NOUN when prev tag is  NOUN
Correcting ::  betting  to NOUN when prev tag is  VERB
Correcting ::  providers  to NOUN when prev tag is  VERB
Correcting ::  pregnant  to NOUN when prev tag is  ADP
Correcting ::  Syracuse  to NOUN when prev tag is  ADP
Correcting ::  DyDee  to NOUN when prev tag is  .
Correcting ::  stresses  to NOUN when prev tag is  VERB
Correcting ::  awareness  to NOUN when prev tag is  ADJ
Correcting ::  classed  to NOUN when prev tag is  NOUN
Correcting ::  cargo  to NOUN when prev tag is  ADP
Correcting ::  therefore  to NOUN when prev tag is  CONJ
Correcting ::  chalk  to NOUN when prev tag is  ADP
Correcting ::  touched  t

Correcting ::  defying  to NOUN when prev tag is  X
Correcting ::  generalizations  to NOUN when prev tag is  VERB
Correcting ::  bundling  to NOUN when prev tag is  X
Correcting ::  GOODY  to NOUN when prev tag is  .
Correcting ::  PRODUCTS  to NOUN when prev tag is  NOUN
Correcting ::  11.5  to NUM
Correcting ::  *T*-203  to NOUN when prev tag is  PRON
Correcting ::  Airways  to NOUN when prev tag is  ADJ
Correcting ::  FIRST  to NOUN when prev tag is  .
Correcting ::  CAMPAIGN  to NOUN when prev tag is  NOUN
Correcting ::  hugging  to NOUN when prev tag is  ADV
Correcting ::  cleanup  to NOUN when prev tag is  ADJ
Correcting ::  burn  to NOUN when prev tag is  CONJ
Correcting ::  cleaner-burning  to NOUN when prev tag is  NOUN
Correcting ::  fuels  to NOUN when prev tag is  NOUN
Correcting ::  rigors  to NOUN when prev tag is  DET
Correcting ::  glory  to NOUN when prev tag is  PRON
Correcting ::  faded  to NOUN when prev tag is  VERB
Correcting ::  yellow  to NOUN when prev tag is 

Correcting ::  16,072  to NUM
Correcting ::  Marion  to NOUN when prev tag is  .
Correcting ::  Stewart  to NOUN when prev tag is  NOUN
Correcting ::  Spitler  to NOUN when prev tag is  NOUN
Correcting ::  18,444  to NUM
Correcting ::  *T*-95  to NOUN when prev tag is  PRON
Correcting ::  306  to NUM
Correcting ::  rang  to NOUN when prev tag is  NOUN
Correcting ::  313  to NUM
Correcting ::  Brunswick  to NOUN when prev tag is  NOUN
Correcting ::  clothing  to NOUN when prev tag is  CONJ
Correcting ::  expense  to NOUN when prev tag is  NOUN
Correcting ::  exceeds  to NOUN when prev tag is  NOUN
Correcting ::  chunk  to NOUN when prev tag is  ADJ
Correcting ::  consistent  to NOUN when prev tag is  VERB
Correcting ::  wedded  to NOUN when prev tag is  VERB
Correcting ::  *T*-213  to NOUN when prev tag is  DET
Correcting ::  sacrificing  to NOUN when prev tag is  X
Correcting ::  *T*-140  to NOUN when prev tag is  ADP
Correcting ::  *T*-173  to NOUN when prev tag is  DET
Correcting :: 

[[(',', '.'), (('refuses', 'NOUN'), ('refuses', 'VERB'))],
 [('the', 'DET'), (('wound', 'VERB'), ('wound', 'NOUN'))],
 [('and', 'CONJ'), (('restructuring', 'NOUN'), ('restructuring', 'VERB'))],
 [('are', 'VERB'), (('betting', 'NOUN'), ('betting', 'VERB'))],
 [('health', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('and', 'CONJ'), (('that', 'DET'), ('that', 'ADP'))],
 [('of', 'ADP'), (('pregnant', 'NOUN'), ('pregnant', 'ADJ'))],
 [('the', 'DET'), (('right', 'NOUN'), ('right', 'ADJ'))],
 [('this', 'DET'), (('minute', 'ADJ'), ('minute', 'NOUN'))],
 [('marketing', 'NOUN'), (('push', 'VERB'), ('push', 'NOUN'))],
 [('push', 'NOUN'), (('stresses', 'NOUN'), ('stresses', 'VERB'))],
 [('vehicles', 'NOUN'), (('classed', 'NOUN'), ('classed', 'VERB'))],
 [('carry', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('and', 'CONJ'), (('therefore', 'NOUN'), ('therefore', 'ADV'))],
 [('chalk', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('first', 'ADV'), (('touched', 'NOUN'), ('touched', 'VER

In [11]:
# Read the Test Sentences 
test_file = pd.read_csv('Test_sentences.txt', sep="\t", header = None)
test_sent_list = test_file[0].tolist()
test_run = []
for sent in test_sent_list:
    test_run.append(nltk.pos_tag(nltk.word_tokenize(sent), 'universal'))  
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for tup in test_run_base]
test_tagged_words

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since',
 '2011',
 'and',
 'on',
 'tablets',
 'since',
 '2013',
 '.',
 'Google',
 'and',
 'Twitter',
 'made',
 'a',
 'deal',
 'in',
 '2015',
 'that',
 'gave',
 'Google',
 'access',
 'to',
 'Twitter',
 "'s",
 'firehose',
 '.',
 'Twitter',
 'is',
 'an',
 'online',
 'news',
 'and',
 'social',
 'networking',
 'service',
 'on',
 'which',
 'users',
 'post',
 'and',
 'interact',
 'with',
 'messages',
 'known',
 'as',
 'tweets',
 '.',
 'Before',
 'entering',
 'politics',
 ',',
 'Donald',
 'Trump',
 'was',
 'a',
 'domineering',
 'businessman',
 'and',
 'a',
 'television',
 'personality',
 '.',
 'The',
 '2018',
 'FIFA',
 'World',
 'Cup',
 'is',
 'the',
 '21st',
 'FIFA',
 'World',
 'Cup',
 ',',
 'an',
 'international',
 'football',
 'tournament',
 'contested',
 'once',
 'every',
 'four',
 'years',
 '.'

In [12]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print ("Time Taken with Vanila Viterbi on Test Data ::", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_vanila = len(check)/len(tagged_seq)
print ("Accuracy ::", accuracy_vanila)
incorrect_tagged_cases_vanila = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_vanila

Time Taken with Vanila Viterbi on Test Data :: 14.898024797439575
Accuracy :: 0.7569060773480663


[[('.', '.'), (('Android', '.'), ('Android', 'NOUN'))],
 [('by', 'ADP'), (('Google', '.'), ('Google', 'NOUN'))],
 [('.', '.'), (('Android', '.'), ('Android', 'NOUN'))],
 [('best-selling', 'ADJ'), (('OS', '.'), ('OS', 'NOUN'))],
 [('OS', 'NOUN'), (('worldwide', '.'), ('worldwide', 'NOUN'))],
 [('on', 'ADP'), (('smartphones', '.'), ('smartphones', 'NOUN'))],
 [('since', 'ADP'), (('2011', '.'), ('2011', 'NUM'))],
 [('since', 'ADP'), (('2013', '.'), ('2013', 'NUM'))],
 [('.', '.'), (('Google', '.'), ('Google', 'NOUN'))],
 [('and', 'CONJ'), (('Twitter', '.'), ('Twitter', 'NOUN'))],
 [('in', 'ADP'), (('2015', '.'), ('2015', 'NUM'))],
 [('gave', 'VERB'), (('Google', '.'), ('Google', 'NOUN'))],
 [('to', 'PRT'), (('Twitter', '.'), ('Twitter', 'NOUN'))],
 [('Twitter', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [("'s", 'PRT'), (('firehose', '.'), ('firehose', 'NOUN'))],
 [('.', '.'), (('Twitter', '.'), ('Twitter', 'NOUN'))],
 [('an', 'DET'), (('online', '.'), ('online', 'ADJ'))],
 [('and', 'CONJ

In [13]:
# tagging the test sentences using modified Viterbi
start = time.time()
tagged_seq = ViterbiNew(test_tagged_words)
end = time.time()
difference = end-start
print ("Time Taken with Modified Viterbi on Test Data ::", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_new = len(check)/len(tagged_seq)
print ("Accuracy ::", accuracy_new)
incorrect_tagged_cases_new = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_new

Correcting ::  Android  with most common tag  NOUN
Correcting ::  Google  to NOUN when prev tag is  ADP
Correcting ::  Android  to NOUN when prev tag is  .
Correcting ::  OS  to NOUN when prev tag is  ADJ
Correcting ::  worldwide  to NOUN when prev tag is  NOUN
Correcting ::  smartphones  to NOUN when prev tag is  ADP
Correcting ::  2011  to NUM
Correcting ::  2013  to NUM
Correcting ::  Google  to NOUN when prev tag is  .
Correcting ::  Twitter  to NOUN when prev tag is  CONJ
Correcting ::  2015  to NUM
Correcting ::  Google  to NOUN when prev tag is  VERB
Correcting ::  Twitter  to NOUN when prev tag is  PRT
Correcting ::  firehose  to NOUN when prev tag is  PRT
Correcting ::  Twitter  to NOUN when prev tag is  .
Correcting ::  online  to NOUN when prev tag is  DET
Correcting ::  interact  to NOUN when prev tag is  CONJ
Correcting ::  messages  to NOUN when prev tag is  ADP
Correcting ::  tweets  to NOUN when prev tag is  ADP
Correcting ::  domineering  to NOUN when prev tag is  DET


[[('2015', 'NUM'), (('that', 'ADP'), ('that', 'DET'))],
 [('an', 'DET'), (('online', 'NOUN'), ('online', 'ADJ'))],
 [('a', 'DET'), (('domineering', 'NOUN'), ('domineering', 'ADJ'))],
 [('tournament', 'NOUN'), (('contested', 'NOUN'), ('contested', 'VERB'))],
 [('the', 'DET'), (('11th', 'ADJ'), ('11th', 'NUM'))],
 [('.', '.'), (('Show', 'NOUN'), ('Show', 'VERB'))],
 [('to', 'PRT'), (('Atlanta', 'NOUN'), ('Atlanta', 'VERB'))],
 [('would', 'VERB'), (('like', 'ADP'), ('like', 'VERB'))],
 [('.', '.'), (('Show', 'NOUN'), ('Show', 'VERB'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('NASA', 'NOUN'), (('invited', 'NOUN'), ('invited', 'VERB'))],
 [('to', 'PRT'), (('experience', 'NOUN'), ('experience', 'VERB'))]]

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [14]:
# Improvement in acuracy over vanilla Viterbi Algorith
accuracy_improvement = round(((accuracy_new - accuracy_vanila)/accuracy_vanila)*100, 2)
print ("Accuracy Improvement = ", accuracy_improvement, "%")

Accuracy Improvement =  23.36 %


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [15]:
# List down at least three cases from the sample test file 
# (i.e. unknown word-tag pairs) which were incorrectly tagged 
# by the original Viterbi POS tagger and got corrected after your modifications.

# Case 1
# Numbers
# Correcting ::  2011  to NUM
# Correcting ::  2013  to NUM
# checkNumberTagger has correctly tagged these as NUM


# Case 2
# Prev Tag
# Correcting ::  Google  to NOUN when prev tag is  VERB
# Correcting ::  Twitter  to NOUN when prev tag is  PRT
# checkBiagramTagger has used prev tag frequeny to tag the correct word

# Case 3
# Most Frequent Tag 
# Correcting ::  Android  with most common tag  NOUN
# checkLexicanTagger has used most frequent tag instead of random tag
